In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mayankverma341","key":"0725e8cbd3bd31dcd476f7a1a84d27bb"}'}

In [6]:
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d senju14/ocr-dataset-of-multi-type-documents

Dataset URL: https://www.kaggle.com/datasets/senju14/ocr-dataset-of-multi-type-documents
License(s): MIT
 99% 5.09G/5.16G [01:21<00:04, 17.7MB/s]
100% 5.16G/5.16G [01:21<00:00, 68.2MB/s]


In [8]:
import os
dataset = "ocr-dataset-of-multi-type-documents"
zip_file = f"{dataset}.zip"
destination_dir = f"/content/{dataset}"
!unzip -q $zip_file -d $destination_dir
!rm $zip_file

In [9]:
!apt install tesseract-ocr
!pip install pytesseract
!pip install jiwer
!pip install easyocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 31.5 MB/s eta 0:00:00


In [13]:
import pytesseract
from jiwer import cer, wer
from tqdm import tqdm
import json
import cv2
import pandas as pd
import numpy as np
import easyocr

In [14]:
reader = easyocr.Reader(['en'],gpu = True)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [47]:
def preprocess_image(image_path):
  img = cv2.imread(image_path)
  gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  return img

In [25]:
dataset_path= "/content/ocr-dataset-of-multi-type-documents/invoice/test/images"
dataset_path_json= "/content/ocr-dataset-of-multi-type-documents/invoice/test/annotations"

files = [f for f in os.listdir(dataset_path)]

def get_ground_truth_from_json(json_file):

    try:
        with open(json_file, 'r') as f:
            data = json.load(f)

        if 'ocr_boxes' in data:
            text = [box["text"] for box in data["ocr_boxes"]]
            full_text = " ".join(text).strip()
            return full_text

        else:
            return ""
    except Exception as e:
        # print(f"Error reading JSON {json_file}: {e}")
        return ""

In [48]:
results = []

for filename in tqdm(files, desc="Processing Images"):
    image_path = os.path.join(dataset_path, filename)
    base_name = os.path.splitext(filename)[0]
    json_path = os.path.join(dataset_path_json, base_name + ".json")

    if not os.path.exists(json_path):
        continue

    ground_truth = get_ground_truth_from_json(json_path)

    if not ground_truth.strip():
        continue

    processed_img = preprocess_image(image_path)
    if processed_img is None:
        continue

    predicted_text = reader.readtext(processed_img)

    full_text = []
    for bbox,text, conf in predicted_text:
      full_text.append(text)
      pred_clean = " ".join(full_text)


    gt_clean = " ".join(ground_truth.split())


    try:
      error_cer = cer(gt_clean.upper(), pred_clean.upper())
      error_wer = wer(gt_clean.upper(), pred_clean.upper())

    except Exception as e:
      # print(e)
      continue

    results.append({"file_name":filename,"ocr_text":pred_clean.upper(),"ground_truth":gt_clean.upper(),"cer":error_cer,"wer":error_wer})


df = pd.DataFrame(results)
    # df.head()

Processing Images: 100%|██████████| 98/98 [02:09<00:00,  1.32s/it]


In [51]:
avg_cer = df["cer"].mean()
avg_wer = df["wer"].mean()
count = df.shape[0]
print("\n" + "="*30)
print(f"Evaluation Complete on {count} images")
print(f"Average CER: {avg_cer:.4f}")
print(f"Average WER: {avg_wer:.4f}")
print("="*30)



Evaluation Complete on 98 images
Average CER: 0.2192
Average WER: 0.5415


In [52]:
df.head()

,file_name,ocr_text,ground_truth,cer,wer
0,X51005675095.jpg,3 '8 0OV SLAKIS DJULAMAS SUN VICTCRY SON BHD (...,SOLARIS DUTAMAS SUN VICTORY SDN BHD (NO. 19357...,0.234649,0.826667
1,X51007103610.jpg,"UOC 8 MK D.I.Y _ (M) SDN BHD (C0,REG 860671-D ...",MR. D.I.Y. (M) SDN BHD (CO.REG :860671-D) LOT ...,0.364780,0.975610
2,X51005675914.jpg,"BEYOND BROTHERS HARDWARE (872981-V) LOT 1-0-2,...","BEYOND BROTHERS HARDWARE (872981-V) LOT 1-0-2,...",0.292411,0.666667
3,X51005433533.jpg,F3O 41 PERNIAGAAN ZHENG HUI JM0325955-V NO.59 ...,PERNIAGAAN ZHENG HUI JM0325955-V NO.59 JALAN P...,0.172158,0.461883
4,X51007846366.jpg,"AOBOLT AEON CO , (M) BHD (126926-H) 3RD FLR , ...","AEON CO. (M) BHD (126926-H) 3RD FLR, AEON TAMA...",0.349593,0.640845
